## Global Variables

In [16]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt



#global variables==============================================
canvas_grid_x_num = 32
canvas_grid_y_num = 32
canvas_x = 2716400/2000
canvas_y = 2650880/2000
grid_width =canvas_x/(canvas_grid_x_num)
grid_height=canvas_y/(canvas_grid_y_num)

hard_macro_num = 16 #adjacency index 0~15
soft_macro_num = 4800 #adjacency index 16~4815
pin_num = 1211 #adjacency index 4816~6026
std_num = 191987


std_width = 8
std_height = 1.2

soft_macro_area = std_width * std_height *40
soft_macro_size = math.sqrt(soft_macro_area)

#coulomb 
k = 8.9876*(10 ** 9)

partition_number = 4800
#=================================================================

## Parsing

In [17]:
def read_cells(filename):

  #각 macro들의 이름 및 크기 정보 가진 딕셔너리
  hard_macro_info={}
  std_info={}

  #각 macro들의 이름만 가진 배열
  hard_macro_name=[]
  std_name=[]

  macro_read_en=0

  #lef 파일로부터 넷리스트에 포함된 macro들의 정보 읽어오기=========================

  with open(filename + '.lef') as n:

    for num, line in enumerate(n):

      if '#' in line:
        continue


      if 'MACRO' in line: #새로운 macro 종류 등장
        macro_read_en=1 
        ismacro = 0 #1일경우 hard macro, 0일경우 std
        data = line.split()
        macro_name = data[1]

      #macro 정보가 아닌 곳에서 size읽어오면 안되므로 enable 신호 사용
      if macro_read_en: 

        #macro가 hard macro인지 std인지 분류====================
        if 'CLASS' in line: 
          data = line.split()
          if(data[1]=='BLOCK'):
            ismacro = 1 #hard macro
          else:
            ismacro = 0 #standard cell
        #=========================================================

        
        elif 'BY' in line:
          data = line.split()
          macro_width = data[1]
          macro_height = data[3]

          if(ismacro): #hard macro일 경우
            hard_macro_info[macro_name] = {'width' : float(macro_width), 'height' : float(macro_height)}

            hard_macro_name.append(macro_name)

          else: #std일 경우
            try:
              std_info[macro_name] =  {'width' : float(macro_width), 'height' : float(macro_height)}


            except ValueError:
              std_info[macro_name] = {'width' : float(data[2]), 'height' : float(data[4])}


            std_name.append(macro_name)

  #=====================================================================================


  #def파일로부터 핀 읽어오기============================================================
  
  pins={}
  read_pin_en=0
  pin_adjacency_index = hard_macro_num+soft_macro_num
  with open(filename + '.def') as n:
    for num, line in enumerate(n):
      #pin read enable==================
      if 'END PINS' in line:
        read_pin_en = 0
      elif 'PINS' in line:
        read_pin_en = 1
      #==================================
      
      
      if(read_pin_en):
        if '- pin' in line:
          data = line.split()
          pin_name = data[1] #pin의 이름
          pin_connected_net = data[4] #pin이 어떤 net에 연결되어 있는지
        elif 'PLACED' in line:
          data = line.split()

          #um단위
          pin_x = int(data[3])/2000
          pin_y = int(data[4])/2000

          #(0 0) ( 2716400 0 ) ( 0 2650880 ) ( 2716400 2650880 )와 같이 꼭짓점에는 핀이 없당

          #핀리스트에 추가
          pins[pin_name] = {'connected_net' : pin_connected_net, 'adjacency_index':pin_adjacency_index, 'x':pin_x, 'y':pin_y}
          pin_adjacency_index = pin_adjacency_index+1



  #=====================================================================================


        
  #def 파일로부터 component의 이름 읽어오기=======================================

  hard_macros = {}
  stds = {}

  #initialize
  hard_macro_adjacency_index = 0
  
  read_component_en=0
  std_hmetis_index=1 

  with open(filename + '.def') as n:
    for num, line in enumerate(n):
      #component read enable=============
      if 'END COMPONENTS' in line:
        read_component_en = 0
      elif 'COMPONENTS' in line:
        read_component_en = 1
      #==================================

      if(read_component_en):
        if('- ') in line:
          data = line.split()
          component_name = data[1] #각 component의 이름
          component_macro_name = data[2] #각 component가 어떤 매크로인지지


          #component_macro_name이 hard_macro_name에 속해있는 경우
          if component_macro_name in hard_macro_name:
            random_place_x = random.random() * canvas_x
            random_place_y = random.random() * canvas_y
            hard_macros[component_name] = {'connected_nets' : [], 'adjacency_index':hard_macro_adjacency_index, 'x': random_place_x, 'y':random_place_y}
            hard_macro_adjacency_index = hard_macro_adjacency_index + 1
          #component_macro_name이 std_name에 속해있는 경우
          elif component_macro_name in std_name:
            stds[component_name] = {'connected_nets' : [], 'hmetis_index': std_hmetis_index}
            std_hmetis_index=std_hmetis_index+1


  return hard_macros, stds, pins, hard_macro_name, std_name


def read_nets(filename, hard_macros, stds, pins, hard_macro_name, std_name):

  read_net_en=0
  net_list = {}
  pin_adjacency_index = hard_macro_num + soft_macro_num
  
  with open(filename + '.def') as n:
    for num, line in enumerate(n):

      #net read enable===================
      if 'END NETS' in line:
        read_net_en = 0
      elif 'NETS' in line:
        read_net_en = 1
      #==================================

      if(read_net_en):
        #새로운 net 등장
        if '-' in line:
          data = line.split()
          net_name = data[1] #net의 이름
          
          net_list[net_name] = {'connected_stds': [], 'connected_hard_macros' :[], 'connected_soft_macros' :[],'connected_pins' :[], 'connected_adjacency_indices':[]}
        else:
          data = line.split()
          #해당 net에 포함된 pin 및 component들 추가====
          for component_name in data: 
            if 'pin' in component_name: #component가 pin일 경우
              net_list[net_name]['connected_pins'].append(component_name)
              net_list[net_name]['connected_adjacency_indices'].append(pins[component_name]['adjacency_index'])
              

            elif 'inst' in component_name:
              if component_name in hard_macros: #component가 hard macro인 경우
                hard_macros[component_name]['connected_nets'].append(net_name)  #cell list 내의 net update
                net_list[net_name]['connected_hard_macros'].append(component_name)  #net list update
                net_list[net_name]['connected_adjacency_indices'].append(hard_macros[component_name]['adjacency_index'])



              elif component_name in stds: #component가 std인 경우
                stds[component_name]['connected_nets'].append(net_name)  #cell list 내의 net update
                net_list[net_name]['connected_stds'].append(component_name)  #net list update

          #==============================================
  

  return net_list


def make_HGraphFile(filename, net_list, stds):
  

  f=open("./netlist/HGraphFile.txt", 'w')

  #write the information of graph at the top
  info_data="%d %d\n" % (len(net_list), len(stds)+1)
  f.write(info_data) 

  
  for net_name in net_list:
    data=""
    for std_name in net_list[net_name]['connected_stds']:
      data = data + str(stds[std_name]['hmetis_index']) + ' '
    data=data+'\n'
    if(data=='\n'):
      data = "%d\n" % (len(stds)+1)  #hard macro만 포함된  net의 모든 hard macro는 191988번째 std로 가정(hmetis 오류 방지용)
      
    f.write(data)
    
  f.close()

In [18]:
def make_softmacros(partition_num, stds, hard_macros):
  
  
  #빠른 서치를 위해 hmetis_indices 만들기==============================
  hmetis_indices_std_name = [i for i in range(len(stds)+1)]
  
  i=1
  for std_name in stds:
    if(stds[std_name]['hmetis_index']==i):
      hmetis_indices_std_name[i]=std_name
      i = i+1
      
  #===================================================================
  
  #soft_macros 딕셔너리 만들기=========================================
  soft_macros = {}
  soft_macro_count = 0
  soft_macro_adjacency_index = hard_macro_num
  
  print("making soft macros start!")
  with open('./netlist/HGraphFile.hgr.part.'+str(partition_num)) as n:

    for num,line in enumerate(n):
      data = line.split()
      soft_macro_hmetis_name = data[0]

      #Hmetis 시 마지막에 가상의 std cell 넣어두었으므로 실제 std cell이 아니면 읽어오지 않는다
      if(num>=std_num):
        break
      
      else:
        #hmetis 결과 안에 각 std가 몇 번째 softmacro인지 작성되어 있으므로 이 번호를 이용해 소프트매크로의 이름 만들기
        soft_macro_name = 'softmacro' + soft_macro_hmetis_name 
        
        if(soft_macro_name in soft_macros):
          #num번째(num은 0부터 시작) 줄에 작성된 정보는 num+1번 hmetis index를 갖는 std cell이 속한 soft macro를 나타낸다. 
          std_hmetis_index = num+1
          
          #소프트매크로 안에 포함된 스탠다드 셀들
          soft_macros[soft_macro_name]['clustered_stds'].append(hmetis_indices_std_name[std_hmetis_index])

          #소프트매크로 안에 포함된 스탠다드 셀들 정보를 이용해 connected net 정보 추가
          for std_name in soft_macros[soft_macro_name]['clustered_stds']:
              soft_macros[soft_macro_name]['connected_nets'] = soft_macros[soft_macro_name]['connected_nets'] + stds[std_name]['connected_nets']

          
        
        #처음 등장한 softmacro일 경우
        else:
          soft_macros[soft_macro_name] = {'connected_nets': [], 'adjacency_index': soft_macro_adjacency_index, 'clustered_stds':[], 'size':soft_macro_size, 
                                          'x' : 0, 'y' : 0, 'ePlace_grid_x' : 0, 'ePlace_grid_y' : 0, 
                                          'attraction_x' : 0, 'attraction_y' : 0, 'repulsion_x':0, 'repulsion_y' : 0, 'electric_force_x':0, 'electric_force_y' : 0}
          soft_macro_count = soft_macro_count+1
          soft_macro_adjacency_index = soft_macro_adjacency_index + 1
  #=================================================================================
  
  
  #만들어진 소프트 매크로들의 정보를 넷리스트에도 추가하기===============================
  for soft_macro_name in soft_macros:
    for net_name in soft_macros[soft_macro_name]['connected_nets']:
      net_list[net_name]['connected_soft_macros'].append(soft_macro_name)
      
      if(soft_macros[soft_macro_name]['adjacency_index'] in net_list[net_name]['connected_adjacency_indices']):
        continue
      else:
        net_list[net_name]['connected_adjacency_indices'].append(soft_macros[soft_macro_name]['adjacency_index'])
  #==================================================================================


  print("making soft macros finish!")
  
  return soft_macros

def make_adjacency_matrix(net_list, hard_macros, soft_macros, pins, total_data_num):
  

  adjacency_matrix = [[0 for i in range(total_data_num)] for j in range(total_data_num)]
  
  print("making adjacency matrix start!")
  for net_name in net_list:
    for index_x in net_list[net_name]['connected_adjacency_indices']:
      for index_y in net_list[net_name]['connected_adjacency_indices']:
        adjacency_matrix[index_x][index_y] = 1


  for i in range(len(adjacency_matrix)):
    adjacency_matrix[i][i] = 0
  
  return adjacency_matrix

## ePlace

In [22]:
#ePlace===========================================================
def ePlace(adjacency_matrix, hard_macros, soft_macros, pins, net_list):
    
    
    #ePlace에서 사용되는 변수들 정리==========================================
    ePlace_grid_num = 64 #칩 캔버스를 128*128 그리드로 분할
    iteration_num = 50 #ePlace iteration 횟수
    
    #아래의 scale은 um단위
    
    ePlace_grid_width = 1360/ePlace_grid_num
    ePlace_grid_height = 1330/ePlace_grid_num
    ePlace_grid_area = ePlace_grid_width * ePlace_grid_height
    

    #parameters
    time_interval = 1
    soft_macro_charge = 1
    hard_macro_charge = 100
    pin_charge = 10
    soft_macro_mass = 1
    
    repulsion_weight = 1
    attraction_weight = 5
    
    total_data_num = len(adjacency_matrix)

    #======================================================================
    
    #grid decomposition
    ePlace_grid =  [[ {'included_soft_macros' : [], 'charge_density':0, 'electric_field_x' : 0, 'electric_field_y' 'potential_energy':0} for i in range(ePlace_grid_num)] for j in range(ePlace_grid_num)]

    #eplace 시작 전 모든 소프트 매크로 랜덤 배치====================================
    
    x_places = np.random.rand(total_data_num)
    y_places = np.random.rand(total_data_num)
    
    #핀과 하드 매크로 크기는 정해져 있으므로 해당 데이터 가져 온다=========
    place_index = hard_macro_num + soft_macro_num
    for pin_name in pins:
        x_places[place_index] = pins[pin_name]['x']
        y_places[place_index] = pins[pin_name]['y']
        place_index = place_index + 1
        
    place_index = 0
    for hard_macro_name in hard_macros:
        x_places[place_index] = hard_macros[hard_macro_name]['x']
        y_places[place_index] = hard_macros[hard_macro_name]['y']
        place_index = place_index + 1
    #==================================================================
    
    #=============================================================================
    
    
    #charge의 곱 행렬 만들기 -> 힘을 받을 셀은 행의 index, 힘을 줄 셀은 열의 index들
    charges = np.zeros(total_data_num)
    charges[0:hard_macro_num] = hard_macro_charge
    charges[hard_macro_num:hard_macro_num+soft_macro_num] = soft_macro_charge
    charges[hard_macro_num+soft_macro_num:total_data_num] = pin_charge
    
    charge_multiplication = np.eye(total_data_num)
    
    for row_index in range(total_data_num):
        charge_multiplication[row_index, :] = charges[row_index] * charges
    
    
    #r_x 행렬 선언
    r_x_for_attraction = np.zeros(total_data_num)
    r_y_for_attraction = np.zeros(total_data_num)

    
    attraction_force = np.zeros(total_data_num)
    repulstion_force = np.zeros(total_data_num)
    
    #ePlace의 iteration ================================================
    for iter in range(iteration_num):

        print("iteration ", iter, "=====================================")
        
        #모든 grid의 정보 초기화 및 charge density 계산==========
        print("calculate charge density")
        for row in ePlace_grid:
            for ePlace_grid_to_update in row:

                 
                #이 bin의 charge density 계산=======
                total_cell_area = len(ePlace_grid_to_update['included_soft_macros']) * soft_macro_area 
                charge_density = soft_macro_charge * total_cell_area/ePlace_grid_area
                ePlace_grid_to_update['charge_density'] = charge_density
                #==================================


        #========================================================



        #모든 그리드의 전기장 계산 =============================
        
        #빠른 연산을 위해 charge density 배열 만들기
        
        ePlace_grid_charge_density = np.zeros(ePlace_grid_num, ePlace_grid_num)
        
        
        
        print("calculate electric field")
        
        
        '''
        for grid_to_look_x in range (0, ePlace_grid_num):
            for grid_to_look_y in range (0, ePlace_grid_num): 
              
             #현재 볼 그리드에 작용하고 있는 전기장
              ePlace_field_x=0
              ePlace_field_y=0
              grid_to_look_charge = ePlace_grid[grid_to_look_x][grid_to_look_y]['charge_density']

              for grid_to_effect_x in range (0, ePlace_grid_num):
                for grid_to_effect_y in range(0,ePlace_grid_num):

                    #x, y방향 전기장==============
                    
                    r_x = (grid_to_look_x - grid_to_effect_x) * ePlace_grid_width * (10 ** -6)
                    r_y = (grid_to_look_y - grid_to_effect_y) * ePlace_grid_height * (10 ** -6)

                    grid_to_effect_charge = ePlace_grid[grid_to_effect_x][grid_to_effect_y]['charge_density']

                    if (r_x==0):
                        partial_electric_field_x = 0
                    else:
                        partial_electric_field_x = k * grid_to_effect_charge * r_x / (abs(r_x) ** 3)
                    
                    if(r_y==0):
                        partial_electric_field_y = 0
                    else:
                        partial_electric_field_y = k * grid_to_effect_charge * r_y / (abs(r_y) ** 3)

                    ePlace_field_x += partial_electric_field_x
                    ePlace_field_y += partial_electric_field_y
                    
                    #=============================

            ePlace_grid[grid_to_look_x][grid_to_look_y]['repulsion_x']=ePlace_field_x * grid_to_look_charge
            ePlace_grid[grid_to_look_x][grid_to_look_y]['repulsion_y']=ePlace_field_y * grid_to_look_charge

        #========================================================
        '''

        #net으로 연결된 셀들끼리는 인력 작용========================
        print("calculate net attraction")
        
        #r_x, r_y 계산
        for row_index in range(total_data_num):
            r_x_for_attraction[row_index, :] = x_places - x_places[row_index] 
            r_y_for_attraction[row_index, :] = y_places - y_places[row_index] 
        
        #0이 분모에 오는 상황 방지하기 위해 대각선 방향 행렬 모두 임의의 수로 설정
        for i in range(total_data_num):
            r_x_for_attraction[i,i] = 1
            r_y_for_attraction[i,i] = 1
            
        
        
        '''
        for net_name in net_list:
           for soft_macro_name in net_list[net_name]['connected_soft_macros']:
                
                
                attraction_x_by_soft_macro = 0
                attraction_x_by_hard_macro = 0
                attraction_x_by_pin = 0
                
                attraction_y_by_soft_macro = 0
                attraction_y_by_hard_macro = 0
                attraction_y_by_pin = 0
                
                
                #소프트 매크로가 다른 소프트 매크로에 의해 받는 총 인력
                for effective_soft_macro_name in net_list[net_name]['connected_soft_macros']:
                    delta_attraction_x, delta_attraction_y = calculate_attraction(soft_macros[soft_macro_name]['x'], soft_macros[effective_soft_macro_name]['x'], soft_macros[soft_macro_name]['y'], soft_macros[effective_soft_macro_name]['y'], soft_macro_charge, soft_macro_charge)
                    attraction_x_by_soft_macro = attraction_x_by_soft_macro + delta_attraction_x
                    attraction_y_by_soft_macro = attraction_y_by_soft_macro + delta_attraction_y
                    
                    
                #소프트 매크로가 하드 매크로에 의해 받는 총 인력
                for effective_hard_macro_name in net_list[net_name]['connected_hard_macros']:
                    delta_attraction_x, delta_attraction_y = calculate_attraction(soft_macros[soft_macro_name]['x'], hard_macros[effective_hard_macro_name]['x'], soft_macros[soft_macro_name]['y'], hard_macros[effective_hard_macro_name]['y'], soft_macro_charge, hard_macro_charge)
                    attraction_x_by_hard_macro = attraction_x_by_hard_macro + delta_attraction_x
                    attraction_y_by_hard_macro = attraction_y_by_hard_macro + delta_attraction_y
                    
                    
                #소프트 매크로가 핀에 의해 받는 총 인력
                for effective_pin_name in net_list[net_name]['connected_pins']:
                    delta_attraction_x, delta_attraction_y = calculate_attraction(soft_macros[soft_macro_name]['x'], pins[effective_pin_name]['x'], soft_macros[soft_macro_name]['y'], pins[effective_pin_name]['y'], soft_macro_charge, pin_charge)
                    attraction_x_by_pin = attraction_x_by_pin + delta_attraction_x
                    attraction_y_by_pin = attraction_y_by_pin + delta_attraction_y
               
               
                soft_macros[soft_macro_name]['attraction_x'] = attraction_x_by_soft_macro + attraction_x_by_hard_macro + attraction_x_by_pin
                soft_macros[soft_macro_name]['attraction_y'] = attraction_y_by_soft_macro + attraction_y_by_hard_macro + attraction_y_by_pin


                #특정 소프트 매크로가 받고 있는 총 힘은 net으로 인해 받고 있는 인력 + 전기장에 의해 받는 힘의 weighted sum
                soft_macros[soft_macro_name]['electric_force_x'] = attraction_weight * soft_macros[soft_macro_name]['attraction_x'] + repulsion_weight * soft_macros[soft_macro_name]['repulsion_x']
                soft_macros[soft_macro_name]['electric_force_y'] = attraction_weight * soft_macros[soft_macro_name]['attraction_y'] + repulsion_weight * soft_macros[soft_macro_name]['repulsion_y']
            '''    
              

        #========================================================
        
        #계산한 힘을 기반으로 소프트 매크로의 위치 이동
        print("move cells")
        move_cells(soft_macros, hard_macros, ePlace_grid, time_interval, soft_macro_mass)
        
        #이동된 소프트 매크로들이 어떤 그리드에 있는지 업데이트
        update_grid_place(soft_macros, ePlace_grid, ePlace_grid_num)
        
        #현재의 상태 display
        display(ePlace_grid, ePlace_grid_num)
        
    #=================================================================
    
    return

def ePlace_grid_init(ePlace_grid_num, soft_macros):
    
    for soft_macro_name in soft_macros:
        random_place_x = random.random() * ePlace_grid_num
        random_place_y = random.random() * ePlace_grid_num
        soft_macros[soft_macro_name]['x'] = random_place_x
        soft_macros[soft_macro_name]['y'] = random_place_y
        
    return
         
def update_grid_place(soft_macros, ePlace_grid, ePlace_grid_num): #모든 cell들이 어떤 그리드에 위치하고 있는지 
    for soft_macro_name in soft_macros:
        soft_macro_x = soft_macros[soft_macro_name]['x']
        soft_macro_y = soft_macros[soft_macro_name]['y']
        soft_macros[soft_macro_name]['ePlace_grid_x'] = soft_macro_x % ePlace_grid_num
        soft_macros[soft_macro_name]['ePlace_grid_y'] = soft_macro_y % ePlace_grid_num
        
    return 

def move_cells(soft_macros, hard_macros, ePlace_grid, time_interval, soft_macro_mass):
    
    #각 그리드 내의 셀들이 어떻게 이동해야 하는지 계산
    
    for soft_macro_name in soft_macros:
        #받고 있는 힘 계산
        ax = soft_macros[soft_macro_name]['electric_force_x']/soft_macro_mass
        delta_x = 0.5 * ax * time_interval * time_interval
        
        ay = soft_macros[soft_macro_name]['electric_force_y']/soft_macro_mass
        delta_y = 0.5 * ay * time_interval * time_interval
        
        #위치 이동 시 경계 조건 고려================================================
        
        soft_macros[soft_macro_name]['x'] = soft_macros[soft_macro_name]['x'] + delta_x
        soft_macros[soft_macro_name]['y'] = soft_macros[soft_macro_name]['y'] + delta_y
        
        
        #캔버스 밖으로 이동 불가
        
        
        #========================================================================
    
    return

def display(soft_macros, hard_macros, pins, ePlace_grid_num):
    
    #grid 그리기
    
    
    
    #핀들 그리기
    pin_x=[]
    pin_y=[]
    
    for pin_name in pins:
        pin_x.append(pins[pin_name]['x'])
        pin_y.append(pins[pin_name]['y'])
        
    
    #하드 매크로 그리기
    hard_macro_x=[]
    hard_macro_y=[]
    
    for hard_macro_name in hard_macros:
        hard_macro_x.append(hard_macros['x'])
        hard_macro_y.append(hard_macros['y'])
    
    #소프트 매크로 그리기
    soft_macro_x=[]
    soft_macro_y=[]
    
    for soft_macro_name in soft_macros:
        soft_macro_x.append(soft_macros['x'])
        soft_macro_y.append(soft_macros['y'])
    
    plt.scatter(soft_macro_x, soft_macro_y)
    return

def calculate_attraction(x1, x2, y1, y2, q1, q2):
    
    rx = (x2-x1) * (10 ** -6)
    ry = (y2-y1) * (10 ** -6)
    
    if (rx==0):
        attraction_x = 0
    else:
        attraction_x = k * (q1 * q2) * (rx) / (abs(rx) ** 3) 
    
    if(ry==0):
        attraction_y=0
    else:
        attraction_y = k * (q1 * q2) * (ry) / (abs(ry) ** 3) 
    
    return attraction_x, attraction_y
#================================================================

## Call Functions

# For Parsing

In [20]:
filename = './netlist/ispd18_test8.input'

#before clustering==========
hard_macros, stds, pins, hard_macro_name, std_name = read_cells(filename)
net_list = read_nets(filename, hard_macros, stds, pins, hard_macro_name, std_name)
make_HGraphFile(filename, net_list,stds)


print("parsing finish!")


#after clustering=========
soft_macros = make_softmacros(partition_number, stds, hard_macros)
total_data_num = len(hard_macros)+len(soft_macros)+len(pins)
adjacency_matrix = make_adjacency_matrix(net_list, hard_macros, soft_macros, pins, total_data_num)


print("making adjacency matrix finish!")

parsing finish!
making soft macros start!
making soft macros finish!
making adjacency matrix start!
making adjacency matrix finish!


# For ePlace

In [23]:

ePlace(adjacency_matrix, hard_macros, soft_macros, pins, net_list)


iteration  0 =====================================
calculate charge density
calculate electric field
calculate net attraction


KeyboardInterrupt: 